<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Import-Tweepy" data-toc-modified-id="Import-Tweepy-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Import Tweepy</a></span></li><li><span><a href="#Analyze-Sentiments-Using-Twitter-Data-and-Tweepy-in-Python" data-toc-modified-id="Analyze-Sentiments-Using-Twitter-Data-and-Tweepy-in-Python-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Analyze Sentiments Using Twitter Data and Tweepy in Python</a></span></li></ul></div>

### Import Tweepy
1. Import the tweepy package
2. Set the authentication credentials
3. Create a new tweepy.API object
4. Use the api object to call the Twitter API

In [ ]:
import tensorflow as tf
print (tf.__version__)

In [ ]:
from utils import sst_binary, lr_results
from encoder import Model

model = Model()

trX, vaX, teX, trY, vaY, teY = sst_binary()
trXt = model.transform(trX[:10])
vaXt = model.transform(vaX[:10])
teXt = model.transform(teX[:10])

clf = lr_results(trXt, trY[:10], vaXt, vaY[:10], teXt, teY[:10])

In [ ]:
# from utils import sst_binary, lr_results
# from encoder import Model

# model = Model()

# trX, vaX, teX, trY, vaY, teY = sst_binary()
# trXt = model.transform(trX)
# vaXt = model.transform(vaX)
# teXt = model.transform(teX)

# clf = lr_results(trXt, trY, vaXt, vaY, teXt, teY)

In [ ]:
from joblib import dump, load
dump(clf, 'logregress_clf.joblib')

In [ ]:
from joblib import load
from encoder import Model

clf = load('logregress_clf.joblib')
model = Model()

In [ ]:
import pandas as pd
import tweepy as tw
import re

# Authenticate to Twitter
consumer_key = "1KbEaA98iLFrLI68cWwZBojKa"
consumer_secret = "UMuEh1JiSu8mVtOpWnDKatg2tXCEf3RGrOyUNBPfC0NgPgUVz0"
access_token = "786958989347033088-k12TcKdTNLTcr16b9W2eAioz07eAwu1"
access_token_secret = "3sU3lCoT6b6Y1Gf1DZPENCaYbbd5B18Dkp8iSYNuLWCSV"

auth = tw.OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token, access_token_secret)
api = tw.API(auth)

tag = "@HarvardBiz"

tweets = api.search(q=tag, lang="en", count=10)
# tweets_text = [tweet.text for tweet in tweets]

def remove_url(txt):
    return " ".join(re.sub("([^0-9A-Za-z \t])|(\w+:\/\/\S+)", "", txt).split())

clean_tweets = [remove_url(tweet.text) for tweet in tweets]

In [ ]:
clean_tweets

In [ ]:
tw_transfrom = model.transform(clean_tweets)
predictions = clf.predict(tw_transfrom)

In [ ]:
predictions

In [ ]:
print("Search tag:", tag)
print()
for i in range(0, 10):
    tw1 = clean_tweets[i]
    sa = predictions[i]
    print(sa, tw1)

In [ ]:
tweets = api.user_timeline(tag, count=10)
for tweet in tweets:
    print(tweet.text)

In [ ]:
tweets1 = [tweet.text for tweet in tweets]

In [ ]:
type(tweets1)

In [ ]:
def remove_url(txt):
    return " ".join(re.sub("([^0-9A-Za-z \t])|(\w+:\/\/\S+)", "", txt).split())

tweets = api.user_timeline(tag, count=10)
tweets_no_urls = [remove_url(tweet.text) for tweet in tweets]
print(tweets_no_urls)

In [ ]:
# create data frame
sa_df = pd.DataFrame(tweets1)
sa_df.columns = ['sentence']
print(sa_df)

In [ ]:
sa_df.insert(0, 'label', '', True)
sa_df['label'] = 0
print(sa_df)

In [ ]:
# sa_df.to_csv (r'C:\Users\zhfen\Desktop\Python\AI1_hw\tweet1.csv', index = None, header=True)

### Analyze Sentiments Using Twitter Data and Tweepy in Python
https://www.earthdatascience.org/courses/earth-analytics-python/using-apis-natural-language-processing-twitter/analyze-tweet-sentiments-in-python/

In [ ]:
import os
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import itertools
import collections

import tweepy as tw
import nltk
from nltk.corpus import stopwords
import re
import networkx
from textblob import TextBlob

import warnings
warnings.filterwarnings("ignore")

sns.set(font_scale=1.5)
sns.set_style("whitegrid")

In [ ]:
# Authenticate to Twitter
consumer_key = "1KbEaA98iLFrLI68cWwZBojKa"
consumer_secret = "UMuEh1JiSu8mVtOpWnDKatg2tXCEf3RGrOyUNBPfC0NgPgUVz0"
access_token = "786958989347033088-k12TcKdTNLTcr16b9W2eAioz07eAwu1"
access_token_secret = "3sU3lCoT6b6Y1Gf1DZPENCaYbbd5B18Dkp8iSYNuLWCSV"

auth = tw.OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token, access_token_secret)

# Create API object
api = tw.API(auth)

try:
    api.verify_credentials()
    print("Authentication OK")
except:
    print("Error during authentication")

In [ ]:
def remove_url(txt):
    return " ".join(re.sub("([^0-9A-Za-z \t])|(\w+:\/\/\S+)", "", txt).split())

In [ ]:
# Create a custom search term and define the number of tweets
search_term = "#climate+change -filter:retweets"

tweets = tw.Cursor(api.search,
                   q=search_term,
                   lang="en",
                   since='2019-11-01').items(100)

# Remove URLs
tweets_no_urls = [remove_url(tweet.text) for tweet in tweets]

In [ ]:
# Analyze Sentiments in Tweets
sentiment_objects = [TextBlob(tweet) for tweet in tweets_no_urls]

sentiment_objects[0].polarity, sentiment_objects[0]

In [ ]:
# Create list of polarity valuesx and tweet text
sentiment_values = [[tweet.sentiment.polarity, str(tweet)] for tweet in sentiment_objects]

sentiment_values[0]

In [ ]:
# Create dataframe containing the polarity value and tweet text
sentiment_df = pd.DataFrame(sentiment_values, columns=["polarity", "tweet"])

# sentiment_df.head()
print(sentiment_df)

In [ ]:
fig, ax = plt.subplots(figsize=(8, 6))

# Plot histogram of the polarity values
sentiment_df.hist(bins=[-1, -0.75, -0.5, -0.25, 0.25, 0.5, 0.75, 1],
             ax=ax,
             color="purple")

plt.title("Sentiments from Tweets on Climate Change")
plt.show()

In [ ]:
# Remove polarity values equal to zero
sentiment_df = sentiment_df[sentiment_df.polarity != 0]

fig, ax = plt.subplots(figsize=(8, 6))

# Plot histogram with break at zero
sentiment_df.hist(bins=[-1, -0.75, -0.5, -0.25, 0.0, 0.25, 0.5, 0.75, 1],
             ax=ax,
             color="purple")

plt.title("Sentiments from Tweets on Climate Change")
plt.show()

In [1]:
%run SA_HuafengZhang.py

Instructions for updating:
non-resource variables are not supported in the long term
Instructions for updating:
dim is deprecated, use axis instead

Search tag: @HarvardBiz

0 RT MoHossain What Happens When Antitrust if any and Protectionism Cycles Collide HarvardBiz Monopoly Trader
1 RT HarvardBiz To build something distinctive in the marketplace you first have to build something distinctive in the workplace
0 RT MoHossain What Happens When Antitrust if any and Protectionism Cycles Collide HarvardBiz Monopoly Trader
0 RT AmazingContent LeadershipLeading a group of people can produce chronic anxiety but many leaders who suffer from anxiety dont
1 The best analysts are lightningfast coders who can surf vast datasets quickly encountering and surfacing potentia
1 RT VALERIEAMALOU A6 Essential A plurality of voices and perspectives is what really helps you find new approaches at work Diversity is
0 At the model companies innovation is a muscle that gets built up over the long haul It takes